In [1]:
import IJulia

# The julia kernel has built in support for Revise.jl, so this is the 
# recommended approach for long-running sessions:
# https://github.com/JuliaLang/IJulia.jl/blob/9b10fa9b879574bbf720f5285029e07758e50a5e/src/kernel.jl#L46-L51

# Users should enable revise within .julia/config/startup_ijulia.jl:
# https://timholy.github.io/Revise.jl/stable/config/#Using-Revise-automatically-within-Jupyter/IJulia-1

# clear console history
IJulia.clear_history()

fig_width = 7
fig_height = 5
fig_format = :retina
fig_dpi = 96

# no retina format type, use svg for high quality type/marks
if fig_format == :retina
  fig_format = :svg
elseif fig_format == :pdf
  fig_dpi = 96
  # Enable PDF support for IJulia
  IJulia.register_mime(MIME("application/pdf"))
end

# convert inches to pixels
fig_width = fig_width * fig_dpi
fig_height = fig_height * fig_dpi

# Intialize Plots w/ default fig width/height
try
  import Plots

  # Plots.jl doesn't support PDF output for versions < 1.28.1
  # so use png (if the DPI remains the default of 300 then set to 96)
  if (Plots._current_plots_version < v"1.28.1") & (fig_format == :pdf)
    Plots.gr(size=(fig_width, fig_height), fmt = :png, dpi = fig_dpi)
  else
    Plots.gr(size=(fig_width, fig_height), fmt = fig_format, dpi = fig_dpi)
  end
catch e
  # @warn "Plots init" exception=(e, catch_backtrace())
end

# Initialize CairoMakie with default fig width/height
try
  import CairoMakie
  
  CairoMakie.activate!(type = string(fig_format))
  CairoMakie.update_theme!(resolution=(fig_width, fig_height))
catch e
    # @warn "CairoMakie init" exception=(e, catch_backtrace())
end
  
# Set run_path if specified
try
  run_path = raw"/home/asmolka/repos/exess/smolkaa.github.io/guide/code_examples/mc_simulation"
  if !isempty(run_path)
    cd(run_path)
  end
catch e
  @warn "Run path init:" exception=(e, catch_backtrace())
end


# emulate old Pkg.installed beahvior, see
# https://discourse.julialang.org/t/how-to-use-pkg-dependencies-instead-of-pkg-installed/36416/9
import Pkg
function isinstalled(pkg::String)
  any(x -> x.name == pkg && x.is_direct_dep, values(Pkg.dependencies()))
end

# ojs_define
if isinstalled("JSON") && isinstalled("DataFrames")
  import JSON, DataFrames
  global function ojs_define(; kwargs...)
    convert(x) = x
    convert(x::DataFrames.AbstractDataFrame) = Tables.rows(x)
    content = Dict("contents" => [Dict("name" => k, "value" => convert(v)) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
elseif isinstalled("JSON")
  import JSON
  global function ojs_define(; kwargs...)
    content = Dict("contents" => [Dict("name" => k, "value" => v) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
else
  global function ojs_define(; kwargs...)
    @warn "JSON package not available. Please install the JSON.jl package to use ojs_define."
  end
end


# don't return kernel dependencies (b/c Revise should take care of dependencies)
nothing


In [2]:
function runMonteCarlo(step::Function, N::Int, p)
    results = []
    for i in 1:N
        push!(results, step(p))
    end
    return results
end

runMonteCarlo (generic function with 1 method)

In [3]:
step(p) = rand(p)
res = runMonteCarlo(step, 100, 10)

100-element Vector{Any}:
 [0.6265432286285892, 0.7095085077968815, 0.6823155591738834, 0.10163419896973369, 0.38515968034381354, 0.6927177229243333, 0.4578292242383073, 0.842379922678032, 0.9071636080378067, 0.0942116063067776]
 [0.352329701380966, 0.6558200456386795, 0.8530692792866649, 0.4662650090792464, 0.22613612467144395, 0.2738941054863864, 0.9669513796779968, 0.16778119038389272, 0.31135047867899435, 0.482015457042126]
 [0.8957760682071086, 0.7206556061804292, 0.445549636886039, 0.004145944054850914, 0.3404603788434213, 0.05674702521223918, 0.9210281434333925, 0.5749588939488433, 0.5523722628463236, 0.23093505543660886]
 [0.5479061332021721, 0.7813028354302568, 0.45767952994693895, 0.823635023667793, 0.05566731909167544, 0.7678347784551042, 0.5766652214760699, 0.4102923287298176, 0.7736011733915418, 0.20386938391286014]
 [0.8030601776751224, 0.3821830383403525, 0.26123454982665506, 0.8941784254199108, 0.21722422694406762, 0.001178319581069065, 0.03039228655560855, 0.08514964883

In [4]:
mean(res)

10-element Vector{Float64}:
 0.5275265569528116
 0.49758838738342304
 0.5261187238829118
 0.5040444135886409
 0.5060254522572045
 0.4927351440862553
 0.4943779417027055
 0.4619091352227478
 0.5246530791568386
 0.5349859422340318

In [5]:
std(res)

10-element Vector{Float64}:
 0.266961615993437
 0.2637777810915042
 0.2968770622209168
 0.28028014272590046
 0.29881595337739586
 0.305157704747627
 0.2962656231369753
 0.3073337698779387
 0.2955110741036773
 0.2847658190298612